# Dataloaders

In [1]:
# Importing libraries
import torchio as tio
import glob
import numpy as np
import random
import os

from collections import OrderedDict
from pathlib import Path

from tqdm import tqdm
import time

import torchio as tio
from torchio.transforms import (RescaleIntensity,RandomFlip,Compose, HistogramStandardization, RandomAffine, RandomNoise, ToCanonical)

from torch.utils.data import DataLoader
import torch
import torch.nn as nn

import matplotlib.pyplot as plt

import pickle

In [2]:
# Define dictionary describing assignment of participants to the groups

# Groups
train_groups=['ABIDE','Athletes','HCP','COBRE','Leipzig']
test_dev_groups=['UoN']
test_groups=['CHIASM']
'''
# CHIASM dobry
# ABIDE dobry
# Athletes dobry
# COBRE dobry
# HCP dobry
# Leipzig good
# MCIC złe
# UoN dobry

# Splits
train_split = 0.85
dev_split = 0.15
test_split = 0.0

# Dictionary with study design
design = {}

design['train']={}
design['dev_train']={}
design['dev_test']={}
design['test']={}

design['train']['all']={}
design['dev_train']['all']={}
design['dev_test']['all']={}
design['test']['all']={}

# Training data
for group in train_groups:
    
    #design['dev_train'][group]={}
    #design['test'][group]={}

    # Idices of all subjects
    ids=[path.split('/')[-2] for path in glob.glob('../../1_Data/1_Input/'+group+'/*/tmp_brain_mask.nii.gz')] 
    # Randomize order
    random.shuffle(ids) 
    # Find split ratios
    train_idx = np.int(np.floor(len(ids)*train_split))
    dev_idx = np.int(np.floor(len(ids)*(train_split+dev_split)))
    
    for i in range(len(ids)):
        
        path_to_folder='../../1_Data/1_Input/'+group+'/'+ids[i]+'/'
        
        files={}
        files['brain']=path_to_folder+'t1w_1mm_iso.nii.gz'
        files['probs']=path_to_folder+'tmp_brain_mask.nii.gz'
        files['chiasm']=path_to_folder+'chiasm.nii.gz'
    
        if i+1<=train_idx:
            design['train']['all'][ids[i]]=files
        elif i+1 > dev_idx:
            design['test']['all'][ids[i]]=files
            #design['test'][group][ids[i]]=files
        else:
            design['dev_train']['all'][ids[i]]=files
            #design['dev_train'][group][ids[i]]=files
        
# Dev data
for group in test_dev_groups:

    # Idices of all subjects
    ids=[path.split('/')[-2] for path in glob.glob('../../1_Data/1_Input/'+group+'/*/tmp_brain_mask.nii.gz')] 
    
    for sub_id in ids:
        
        path_to_folder='../../1_Data/1_Input/'+group+'/'+sub_id+'/'
        
        files={}
        files['brain']=path_to_folder+'t1w_1mm_iso.nii.gz'
        files['probs']=path_to_folder+'tmp_brain_mask.nii.gz'
        files['chiasm']=path_to_folder+'chiasm.nii.gz'
        
        design['dev_test']['all'][sub_id]=files
    
# Test data
for group in test_groups:
    
    #design['test'][group]={}

    # Idices of all subjects
    ids=[path.split('/')[-2] for path in glob.glob('../../1_Data/1_Input/'+group+'/*/tmp_brain_mask.nii.gz')] 
    
    for sub_id in ids:
        
        path_to_folder='../../1_Data/1_Input/'+group+'/'+sub_id+'/'
        
        files={}
        files['brain']=path_to_folder+'t1w_1mm_iso.nii.gz'
        files['probs']=path_to_folder+'tmp_brain_mask.nii.gz'
        files['chiasm']=path_to_folder+'chiasm.nii.gz'
        
        design['test']['all'][sub_id]=files 
        #design['test'][group][sub_id]=files  
        '''

"\n# CHIASM dobry\n# ABIDE dobry\n# Athletes dobry\n# COBRE dobry\n# HCP dobry\n# Leipzig good\n# MCIC złe\n# UoN dobry\n\n# Splits\ntrain_split = 0.85\ndev_split = 0.15\ntest_split = 0.0\n\n# Dictionary with study design\ndesign = {}\n\ndesign['train']={}\ndesign['dev_train']={}\ndesign['dev_test']={}\ndesign['test']={}\n\ndesign['train']['all']={}\ndesign['dev_train']['all']={}\ndesign['dev_test']['all']={}\ndesign['test']['all']={}\n\n# Training data\nfor group in train_groups:\n    \n    #design['dev_train'][group]={}\n    #design['test'][group]={}\n\n    # Idices of all subjects\n    ids=[path.split('/')[-2] for path in glob.glob('../../1_Data/1_Input/'+group+'/*/tmp_brain_mask.nii.gz')] \n    # Randomize order\n    random.shuffle(ids) \n    # Find split ratios\n    train_idx = np.int(np.floor(len(ids)*train_split))\n    dev_idx = np.int(np.floor(len(ids)*(train_split+dev_split)))\n    \n    for i in range(len(ids)):\n        \n        path_to_folder='../../1_Data/1_Input/'+group+

In [3]:
# Save the dictionary
#with open('study_design.pkl', 'wb') as f:
#    pickle.dump(design, f)

# Load the dictionary
with open('study_design.pkl', 'rb') as f:
    design = pickle.load(f)

In [4]:
# Dictionary with all images
subjects_list={}

for group in design.keys():
    subjects_list[group]={}
    
    for dataset in design[group].keys():
        subjects_list[group][dataset]= [tio.Subject(t1=tio.Image(design[group][dataset][sub]['brain'], type=tio.INTENSITY),
                            probs = tio.Image(design[group][dataset][sub]['probs'], type = tio.INTENSITY)) for sub in design[group][dataset].keys()]

In [5]:
# Rescale
rescale = RescaleIntensity((0,1))
# Flip
flip = RandomFlip((0,1,2), flip_probability=0.5, p=0.25)
# Affine transformations
#affine = RandomAffine(degrees=30)

# Composing transforms - rescaling is mandatory, training data is subjected to a range of additional augmentations
transform_train = Compose([rescale, flip]) # leaving out standardization for now
transform_dev = Compose([rescale]) # leaving out standardization for now

In [6]:
# Torchio's (Pytorch's) Dataset
data = {}

for group in subjects_list.keys():
    data[group]={}
    
    for dataset in subjects_list[group]:
        
        if group == 'train':
            data[group][dataset]=tio.SubjectsDataset(subjects_list[group][dataset], transform=transform_train)
        else:
            data[group][dataset]=tio.SubjectsDataset(subjects_list[group][dataset], transform=transform_dev)

In [7]:
# Sampler
patch_size = (24,24,8)
queue_length = 500
samples_per_volume = 5

sampler = tio.data.WeightedSampler(patch_size,'probs')

In [8]:
# Dataloader

dataloader = {}

for group in data.keys():
    dataloader[group]={}
    
    for dataset in data[group]:
        
        dataloader[group][dataset]=DataLoader(tio.Queue(data[group][dataset], queue_length, samples_per_volume, sampler, num_workers=6, shuffle_subjects=True, shuffle_patches=True), batch_size = 25, num_workers=0)


In [9]:
# Testing
'''
num_epochs = 1

model = torch.nn.Identity()

for epoch_index in range(num_epochs):
    for patches_batch in dataloader['dev_train']['all']:
        #print(patches_batch)
        inputs = patches_batch['t1'][tio.DATA]  # key 't1' is in subject
        targets = patches_batch['t1'][tio.DATA]  # key 'brain' is in subject
        logits = model(inputs)  # model being an instance of torch.nn.Module
'''

"\nnum_epochs = 1\n\nmodel = torch.nn.Identity()\n\nfor epoch_index in range(num_epochs):\n    for patches_batch in dataloader['dev_train']['all']:\n        #print(patches_batch)\n        inputs = patches_batch['t1'][tio.DATA]  # key 't1' is in subject\n        targets = patches_batch['t1'][tio.DATA]  # key 'brain' is in subject\n        logits = model(inputs)  # model being an instance of torch.nn.Module\n"

In [10]:
#inputs.shape
'''
fig = plt.figure(figsize=(20, 10))

for i in range(inputs.shape[0]):
    plt.subplot(5,8,i+1)
    plt.imshow(inputs[i,0,:,:,5],cmap='gray');
    
plt.show()
'''

"\nfig = plt.figure(figsize=(20, 10))\n\nfor i in range(inputs.shape[0]):\n    plt.subplot(5,8,i+1)\n    plt.imshow(inputs[i,0,:,:,5],cmap='gray');\n    \nplt.show()\n"

In [11]:
'''import torchio as tio
t1 = tio.ScalarImage('T1w')
t2 = tio.ScalarImage('T2w')
subject = tio.Subject(T1w=t1, T2w=t2)
cp = tio.CropOrPad((512, 512, 408))
subject = tio.Subject(T1w=cp(t1), T2w=cp(t2))
subject.plot(reorient=False)
'''

"import torchio as tio\nt1 = tio.ScalarImage('T1w')\nt2 = tio.ScalarImage('T2w')\nsubject = tio.Subject(T1w=t1, T2w=t2)\ncp = tio.CropOrPad((512, 512, 408))\nsubject = tio.Subject(T1w=cp(t1), T2w=cp(t2))\nsubject.plot(reorient=False)\n"

# Network and parameters

In [12]:
# Try setting CUDA if possible
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu") 
    
print(device)

cuda


In [13]:
# Cropped U-Net copied from Overfitting Model

class UNet(nn.Module):
    
    def __init__(self, in_channels=1, out_channels=1, init_features=10, scaling=2):
        super(UNet, self).__init__()
                
        # Encoding layers
        self.encoder1 = self.unet_block(in_channels, init_features, "enc1")
        self.pool1 = nn.AvgPool3d(kernel_size=2, stride=2, padding=0)
        self.encoder2 = self.unet_block(init_features, init_features*scaling, name='enc2')
        self.pool2 = nn.AvgPool3d(kernel_size=2, stride=2, padding=0)

        # Bottleneck layer
        self.bottleneck = self.unet_block(init_features*scaling, init_features*scaling**2, name='bottleneck')
        
        # Decoding layers (where merge with prevois encoding layers occurs)        
        self.upconv2 = nn.ConvTranspose3d(init_features*scaling**2, init_features*scaling, kernel_size=2, stride=2)
        self.decoder2 = self.unet_block(init_features*scaling, init_features*scaling, name='dec2')
                
        self.upconv1 = nn.ConvTranspose3d(init_features*scaling, init_features, kernel_size=2, stride=2)
        self.decoder1 = self.unet_block(init_features, init_features, name='dec1')
        
        # Final convolution - output equals number of output channels
        self.conv = nn.Conv3d(init_features, out_channels, kernel_size=1) 
        
    def forward(self,x):
        
        # Encoding
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))

        # Bottleneck
        bottleneck = self.bottleneck(self.pool2(enc2))

        # Upconvolving, concatenating data from respective encoding phase and executing UNet block
        dec2 = self.upconv2(bottleneck)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = self.decoder1(dec1)
        
        out_conv = self.conv(dec1)
        
        return torch.sigmoid(out_conv)
    
    def unet_block(self, in_channels, features, name):
        
        return nn.Sequential(OrderedDict([(name+'conv1',nn.Conv3d(in_channels=in_channels, out_channels=features, kernel_size=3, padding=1, bias=False)),
                             (name+'bnorm1', nn.BatchNorm3d(num_features=features)),
                             (name+'relu1', nn.ReLU(inplace=True)),
                             (name+'conv2', nn.Conv3d(in_channels=features, out_channels=features, kernel_size=3, padding=1, bias=False)),
                             (name+'bnorm2', nn.BatchNorm3d(num_features=features)),
                             (name+'relu2', nn.ReLU(inplace=True))])
                            )

    def output_latent_representations(self,x):
        
        print(x.shape)

        # Encoding
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))

        # Bottleneck
        bottleneck = self.bottleneck(self.pool2(enc2))
        
        print(bottleneck.shape)
        
        return bottleneck

In [14]:
#unet = UNet(1,1,2,2) # Size of latent representation = (1/64) * init_features * scaling**2
#unet.to(device)

In [15]:
#print(sum(p.numel() for p in unet.parameters() if p.requires_grad))

In [16]:
# try processing
'''
outputs = unet(inputs.to(device))
outputs = outputs.cpu().detach().numpy()

fig = plt.figure(figsize=(20, 10))

for i in range(outputs.shape[0]):
    plt.subplot(10,10,i+1)
    plt.imshow(outputs[i,0,:,:,5],cmap='gray');
    
plt.show()
'''

"\noutputs = unet(inputs.to(device))\noutputs = outputs.cpu().detach().numpy()\n\nfig = plt.figure(figsize=(20, 10))\n\nfor i in range(outputs.shape[0]):\n    plt.subplot(10,10,i+1)\n    plt.imshow(outputs[i,0,:,:,5],cmap='gray');\n    \nplt.show()\n"

In [17]:
# Criterion
#criterion = DiceLoss()
criterion = nn.MSELoss()

# Number of epochs
n_epochs = 25

In [18]:
#outcome = criterion(inputs, torch.Tensor(outputs))
#print(outcome)

# Training

In [19]:
# Function returning trained model
def train_network(n_epochs, dataloaders, model, optimizer, criterion, device, save_path):
    
    track_train_loss = []
    track_dev_train_loss = []
    track_dev_test_loss = []
    
    valid_loss_min = np.Inf
    
    model.to(device)
        
    for epoch in tqdm(range(1, n_epochs+1)):
        
        # Initialize loss monitoring variables
        train_loss = 0.0
        dev_train_loss = 0.0
        dev_test_loss = 0.0
        
        i=0
        k=0
        j=0
        
        start = time.time()
        
        # Training
        model.train()
        
        for batch in dataloaders['train']['all']:
            
            data = batch['t1']['data'].to(device)
            
            optimizer.zero_grad()
            
            output = model(data)
            loss = criterion(output, data)
            loss.backward()
            
            optimizer.step()
            
            train_loss += loss.item()
            i+=1
            
        track_train_loss.append(train_loss/i)
        
        # Validation on two datasets
        model.eval()
        
        for batch in dataloaders['dev_train']['all']:
            
            data = batch['t1']['data'].to(device)
            
            with torch.no_grad():
                
                output = model(data)
                loss = criterion(output,data)
                
                dev_train_loss += loss.item()
                j+=1
                
        track_dev_train_loss.append(dev_train_loss/j)
        
        
        for batch in dataloaders['dev_test']['all']:
            
            data = batch['t1']['data'].to(device)
            
            with torch.no_grad():
                
                output = model(data)
                loss = criterion(output,data)
                
                dev_test_loss += loss.item()
                k+=1
                
        track_dev_test_loss.append(dev_test_loss/k)
        
        # Print summary of epoch
        duration = time.time() - start

        print('END OF EPOCH: {} \tTraining loss per batch: {:.6f}\tTraining_dev loss per batch: {:.6f}\tTest_dev loss per batch: {:.6f}'.format(epoch, train_loss/i, dev_train_loss/j, dev_test_loss/k))
       
        
        ## Save the model if reached min validation loss
        if dev_train_loss + dev_test_loss < valid_loss_min:
            valid_loss_min = dev_train_loss + dev_test_loss
            torch.save(model.state_dict(),save_path+'optimal_weights')
                        
    # return trained model
    return track_train_loss, track_dev_train_loss, track_dev_test_loss         

In [20]:
#model_parameters=[[64,1],[32,1],[16,2],[16,1],[8,2],[8,1],[4,4],[4,2],[4,1],[2,4],[2,2],[2,1],[1,8],[1,4],[1,2],[1,1]]
model_parameters=[[2,4],[16,1],[4,2],[2,2],[8,2],[4,4],[64,1],[1,8]]

folder='../../1_Data/2_Trained_AE/'

for parameters in model_parameters:
    
    print(parameters)
        
    # Initialize the proper model
    unet = UNet(1,1,parameters[0],parameters[1])
    
    # Optimizer
    optimizer = torch.optim.Adam(params=unet.parameters(), lr=0.005)
    
    # Create output folder
    data_folder = folder+'/'+str(parameters[0])+'_'+str(parameters[1])+'/'
    os.makedirs(data_folder, exist_ok=True)
    
    # Train & save weights
    train_loss, dev_train_loss, dev_test_loss = train_network(n_epochs, dataloader, unet, optimizer, criterion, device, data_folder)
    
    # Save losses
    with open(data_folder+'train_loss.pkl', 'wb') as f:
        pickle.dump(train_loss, f)
        
    with open(data_folder+'dev_train_loss.pkl', 'wb') as f:
        pickle.dump(dev_train_loss, f)
        
    with open(data_folder+'dev_test_loss.pkl', 'wb') as f:
        pickle.dump(dev_test_loss, f)



[2, 4]


  4%|▍         | 1/25 [30:41<12:16:35, 1841.48s/it]

END OF EPOCH: 1 	Training loss per batch: 0.019672	Training_dev loss per batch: 0.006682	Test_dev loss per batch: 0.005112


  8%|▊         | 2/25 [58:35<11:26:37, 1791.19s/it]

END OF EPOCH: 2 	Training loss per batch: 0.006365	Training_dev loss per batch: 0.004665	Test_dev loss per batch: 0.004465


...

END OF EPOCH: 23 	Training loss per batch: 0.003874	Training_dev loss per batch: 0.003343	Test_dev loss per batch: 0.002313


 96%|█████████▌| 24/25 [10:29:07<26:21, 1581.86s/it]

END OF EPOCH: 24 	Training loss per batch: 0.003660	Training_dev loss per batch: 0.004716	Test_dev loss per batch: 0.002492


100%|██████████| 25/25 [10:54:13<00:00, 1570.14s/it]

END OF EPOCH: 25 	Training loss per batch: 0.003753	Training_dev loss per batch: 0.003596	Test_dev loss per batch: 0.002003
